In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import scipy.stats as stats
import nltk.metrics as nm
import editdistance
import re
import matplotlib.pyplot as plt
execfile("utils.py")
%matplotlib inline

Couldn't import dot_parser, loading of dot files will not be possible.


In [22]:
# Load Network with
# 1. Authors
# 2. Dept as node attr
# 3. nPapers as node attr
# 4. year/title as edge attr

G = nx.read_graphml('../save/nber_departments.graphml')

In [24]:
[(u,v,d.keys()) for u,v,d in G.edges_iter(data=True) if 'year' not in d.keys()]
#[(u,v,d.keys()) for u,v,d in G.edges_iter(data=True) if 'Stein' in u or 'Stein' in v]

[]

In [85]:
# Filter on Date Range

init = 0
end = 2016

nodes = G.nodes()
edges = G.edges()

# Create Subgraph with only edges dated to
# the specificed date range
def filterDateRange(G, init, end, datevar='year'):
    
    # Filter Edges for Date Range
    G_out = nx.MultiGraph([(u,v,d) for u,v,d in G.edges_iter(data=True) \
                          if d[datevar] >= init and d[datevar] <= end])
    
    validNodes = G_out.nodes()
    
    # Copy Over Node Attributes
    for attr in ['nPapers', 'dept']:
        validNodes = G_out.nodes()
        d = nx.get_node_attributes(G, attr)
        validAttr = dict([(u,d[u]) for u in d if u in validNodes])
        nx.set_node_attributes(G_out, attr, validAttr)

    return G_out

# Compute number of papers published total (i.e. # of edges) for each author

def nPapers(G, author):
    return len(set([d['title'] for u,v,d in G.edges_iter(data=True) \
                    if u == author or v == author]))

In [182]:
# Develop Success Measure
# (n_internal edges to department / n_total papers)

# Jeremy stein post 2000
author = "ArielPakes"
moveDate = 1999

def computeSuccessMeasure(G, author, moveDate):

    acqDept = nx.get_node_attributes(G, 'dept')[author]

    # Filter for Post-movedate
    G_time = filterDateRange(G, moveDate+1, 9999)

    # Sum weights of ALL edges
    nEdgesTotal = sum([d['weight'] for u,v,d in G_time.edges(author, data=True)])

    # Filter for Acquiering department
    acqDeptNodes = [u for u,d in G_time.nodes_iter(data=True) if 'dept' in d.keys() and d['dept'] == acqDept]
    G_dept = G_time.subgraph(acqDeptNodes)

    # Sum weights of all edges INSIDE acq dept
    nEdges = sum([d['weight'] for u,v,d in G_dept.edges(author, data=True)])

    # Ratio
    measure = nEdges / nEdgesTotal
    
    return (measure, nEdges, nEdgesTotal)

print computeSuccessMeasure(G, author, moveDate)

[('ArielPakes', 'GregoryLewis')]
(0.09999999999994001, 0.333333333333, 3.333333333332)


In [199]:
a = 'DavidCard'
#a = 'PhilippeAghion'
[x for x in G.nodes(data=True) if a in x[0]]

([(u,v,d) for u,v,d in G.edges_iter(data=True) if a == u or a == v])

co = ([(u,v) for u,v,d in G.edges_iter(data=True) if a == u or a == v])
coset = set([c for x in co for c in x])
for c in coset:
    try:
        print (c, nx.get_node_attributes(G, 'dept')[c])
    except:
        print c

LauraGiuliano
ZhuanPei
EthanLewis
('UlrikeMalmendier', u'University of California, Berkeley CA')
('BrianMcCall', u'University of Michigan, Ann Arbor MI')
('PatrickKline', u'University of California, Berkeley CA')
JochenKluve
JohnReenen
PabloIbarraran
('RajChetty', u'Harvard University, Cambridge MA')
PhilipRobins
('StefanoDellaVigna', u'University of California, Berkeley CA')
AndreaWeber
AndrewHildreth
DeanHyslop
('EnricoMoretti', u'University of California, Berkeley CA')
('OrleyAshenfelter', u'Princeton University, Princeton NJ')
PaulineLeung
ThomasLemieux
FerdinandoRegalia
('NicholasBloom', u'Stanford University, Palo Alto CA')
StefanieWolter
FrancescoDevicienti
SaraRica
MartinDooley
CraigOlson
MichaelRansom
JohnDiNardo
WinstonLin
AgataMaida
('RebeccaBlank', u'University of Wisconsin, Madison WI')
PhillipLevine
('DavidLee', u'New York University, New York City NY')
('GordonDahl', u'University of California, San Diego CA')
CarlosDobkin
('DavidCard', u'University of California, Berkele

In [4]:
#def filterEdge(edge, init, end):
#    return nx[edge[0]][edge[1]]

e = G[edges[0][0]][edges[0][1]]
[e[i]['year'] > 2010 for i in e.keys()]

[True, True]

In [5]:
edge = edges[0]
G[edge[0]][edge[1]]

{0: {'jel': u'D0,D8,G1,G3', 'year': 2012},
 1: {'jel': u'D0,D8,G1,G3', 'year': 2012}}

In [14]:
red_fish = set(n for u,v,d in G.edges_iter(data=True)
               if d['year'] > 2005
               for n in (u, v))

In [20]:
nx.MultiGraph([(u,v,d) for u,v,d in G.edges_iter(data=True) if d['year'] > 2005])

In [49]:
x = pd.read_csv('../save/NBER_Paper_Info.csv', delimiter='|')

In [52]:
x[['year','jel']].shape

(17728, 2)

In [57]:
x[['authors','year','jel']].drop_duplicates().shape

(17370, 3)

In [58]:
x[['authors','year','jel']].shape

(17728, 3)

In [59]:
x.groupby('authors')